In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [43]:
avg_rain = pd.read_csv('./data/temp/平均雨量.csv')
price_diff = pd.read_csv('./data/temp/價格乖離.csv')

In [44]:
price_diff = price_diff[['date','year','day_of_year','price_diff']]
price_diff

,date,year,day_of_year,price_diff
0,1996-01-01,1996,1,-8.356861
1,1996-01-02,1996,2,-9.323768
2,1996-01-03,1996,3,-10.459224
3,1996-01-05,1996,5,-9.007560
4,1996-01-06,1996,6,-8.666377
...,...,...,...,...
8757,2025-01-07,2025,7,-33.291719
8758,2025-01-08,2025,8,-38.148728
8759,2025-01-09,2025,9,-32.861248
8760,2025-01-10,2025,10,-41.485029


In [45]:
yearlist = [2016,2017,2018,2019,2020,2021,2022,2023,2024]

In [46]:
price_dict = {}
rain_dict = {}
for i in yearlist:
    price_dict[i] = price_diff[price_diff['year'] == i].reset_index(drop=True)
    rain_dict[i] = avg_rain[avg_rain['year'] == i].reset_index(drop=True)
    price_dict[i] = price_dict[i].drop(columns=['date','year'])
    rain_dict[i] = rain_dict[i].drop(columns=['Date','year'])
# price2016 = price_diff[price_diff['year'] == 2016].reset_index(drop=True)
# rain2016 = avg_rain[avg_rain['year'] == 2016].reset_index(drop=True)
# price2016 = price2016.drop(columns=['date','year'])
# rain2016 = rain2016.drop(columns=['Date','year'])


In [47]:
# price2016

In [48]:
# rain2016

In [55]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense



# 合併數據
main_feature, main_target =[],[]
for i in yearlist[:-2]:
    data = pd.merge(price_dict[i], rain_dict[i], left_on='day_of_year', right_on='dayofyear', how='outer')
    data = data.sort_values('dayofyear').reset_index(drop=True)

    # 填補缺失值
    data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
    data['Precipitation'] = data['Precipitation'].fillna(0)

    # 標準化數據
    scaler = MinMaxScaler()
    data[['price_diff', 'Precipitation']] = scaler.fit_transform(data[['price_diff', 'Precipitation']])

    # 創建序列數據
    sequence_length = 7
    features, targets = [], []
    for i in range(len(data) - sequence_length - 7):
        r_sequence = data['Precipitation'].iloc[i:i + sequence_length].values
        p_sequence = data['price_diff'].iloc[i:i + sequence_length].values
        target = data['price_diff'].iloc[i + sequence_length:i + sequence_length + 7].values
        features.append(np.concatenate([r_sequence, p_sequence]))
        targets.append(target)
    
    main_feature.extend(features)
    main_target.extend(targets)

features = np.array(main_feature)
targets = np.array(main_target)
# data = pd.merge(price2016, rain2016, left_on='day_of_year', right_on='dayofyear', how='outer')
# data = data.sort_values('dayofyear').reset_index(drop=True)

# # 填補缺失值
# data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
# data['Precipitation'] = data['Precipitation'].fillna(0)

# # 標準化數據
# scaler = MinMaxScaler()
# data[['price_diff', 'Precipitation']] = scaler.fit_transform(data[['price_diff', 'Precipitation']])

# # 創建序列數據
# sequence_length = 7
# features, targets = [], []
# for i in range(len(data) - sequence_length - 7):
#     r_sequence = data['Precipitation'].iloc[i:i + sequence_length].values
#     p_sequence = data['price_diff'].iloc[i:i + sequence_length].values
#     target = data['price_diff'].iloc[i + sequence_length:i + sequence_length + 7].values
#     features.append(np.concatenate([r_sequence, p_sequence]))
#     targets.append(target)

# features = np.array(features)
# targets = np.array(targets)

C:\Users\meowm\AppData\Local\Temp\ipykernel_12196\2876959067.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
C:\Users\meowm\AppData\Local\Temp\ipykernel_12196\2876959067.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
C:\Users\meowm\AppData\Local\Temp\ipykernel_12196\2876959067.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
C:\Users\meowm\AppData\Local\Temp\ipykernel_12196\2876959067.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will

In [51]:
features.shape

(2459, 14)

In [54]:
train_size = int(len(features) * 0.8)
X_train, X_test = features[:train_size], features[train_size:]
y_train, y_test = targets[:train_size], targets[train_size:]

# 調整數據形狀
X_train = X_train.reshape((X_train.shape[0], sequence_length, -1))
X_test = X_test.reshape((X_test.shape[0], sequence_length, -1))

# 建立模型
model = Sequential([
    LSTM(64, return_sequences=False, input_shape=(sequence_length, X_train.shape[2])),
    Dense(32, activation='relu'),
    Dense(7)  # 預測7天
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# 訓練模型
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# 評估模型
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")

c:\Users\meowm\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 64)             │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,463 (76.03 KB)

 Trainable params: 19,463 (76.03 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1101 - val_loss: 0.0195
Epoch 2/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0271 - val_loss: 0.0189
Epoch 3/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0250 - val_loss: 0.0176
Epoch 4/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0242 - val_loss: 0.0167
Epoch 5/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0219 - val_loss: 0.0152
Epoch 6/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0196 - val_loss: 0.0148
Epoch 7/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0183 - val_loss: 0.0130
Epoch 8/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0174 - val_loss: 0.0131
Epoch 9/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0173 - val_loss: 0.0139
Epoch 10/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0180 - val_loss: 0.0137
Epoch 11/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0178 - val_loss: 0.0137
Epoch 12/50
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0168 - val_lo

In [56]:
valid_feature, valid_target = [], []
for i in yearlist[-2:]:
    data = pd.merge(price_dict[i], rain_dict[i], left_on='day_of_year', right_on='dayofyear', how='outer')
    data = data.sort_values('dayofyear').reset_index(drop=True)

    # 填補缺失值
    data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
    data['Precipitation'] = data['Precipitation'].fillna(0)

    # 標準化數據
    scaler = MinMaxScaler()
    data[['price_diff', 'Precipitation']] = scaler.fit_transform(data[['price_diff', 'Precipitation']])

    # 創建序列數據
    sequence_length = 7
    features, targets = [], []
    for i in range(len(data) - sequence_length - 7):
        r_sequence = data['Precipitation'].iloc[i:i + sequence_length].values
        p_sequence = data['price_diff'].iloc[i:i + sequence_length].values
        target = data['price_diff'].iloc[i + sequence_length:i + sequence_length + 7].values
        features.append(np.concatenate([r_sequence, p_sequence]))
        targets.append(target)
    
    valid_feature.extend(features)
    valid_target.extend(targets)

valid_features = np.array(valid_feature)
valid_targets = np.array(valid_target)

X_valid = valid_features.reshape((valid_features.shape[0], sequence_length, -1))

loss = model.evaluate(X_valid, valid_targets)
print(f"Valid Loss: {loss}")

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0165 
Valid Loss: 0.017506388947367668


C:\Users\meowm\AppData\Local\Temp\ipykernel_12196\431603359.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
C:\Users\meowm\AppData\Local\Temp\ipykernel_12196\431603359.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['price_diff'] = data['price_diff'].fillna(method='ffill').fillna(method='bfill')
